In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [26]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Rain(inches),Snow(inches),Date
0,0,Bethel,41.37,-73.41,73.40,69,1,5.82,US,clear sky,0.0,0,2020-06-21 02:12:20
1,1,Havre-Saint-Pierre,50.23,-63.60,51.80,87,1,3.36,CA,clear sky,0.0,0,2020-06-21 02:12:20
2,2,Kapaa,22.08,-159.32,84.20,66,40,11.41,US,scattered clouds,0.0,0,2020-06-21 02:12:20
3,3,Alofi,-19.06,-169.92,78.80,78,100,14.99,NU,overcast clouds,0.0,0,2020-06-21 02:10:38
4,4,Puerto Ayora,-0.74,-90.35,69.01,96,82,9.19,EC,broken clouds,0.0,0,2020-06-21 02:07:28


In [27]:
# Get the data types.
city_data_df.dtypes

City_ID                  int64
City                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Country                 object
Current Description     object
Rain(inches)           float64
Snow(inches)             int64
Date                    object
dtype: object

In [28]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [29]:
rain_cities = city_data_df.loc[city_data_df["Rain(inches)"]>0]

In [30]:
snow_cities = city_data_df.loc[city_data_df["Snow(inches)"]>0]

In [31]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [32]:
# Ask the customer if they want Rain or snow
rain_amount = input("Do you want it to be raining? (yes/no)")


Do you want it to be raining? (yes/no)yes


In [33]:
snow_amount = input("Do you want it to be snowing (yes/no)")

Do you want it to be snowing (yes/no)no


In [34]:
if rain_amount == "no" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain(inches)"] ==0) &
                                          (city_data_df["Snow(inches)"] ==0)]  
elif rain_amount == "no" and snow_amount == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain(inches)"] ==0) &
                                          (city_data_df["Snow(inches)"] >=0.0)]
elif rain_amount == "yes" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain(inches)"] > 0.0) &
                                          (city_data_df["Snow(inches)"] == 0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain(inches)"] > 0.0) &
                                          (city_data_df["Snow(inches)"] > 0)]
                                           

In [35]:
filtered_cities_df = filtered_cities_df.dropna()

In [36]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Sangar,RU,73.29,light rain,63.92,127.47,
20,Tecoanapa,MX,82.00,moderate rain,16.52,-98.75,
21,Hearst,CA,62.15,moderate rain,49.68,-83.67,
29,Vangaindrano,MG,65.41,light rain,-23.35,47.60,
91,Panzhihua,CN,82.15,light rain,26.59,101.71,
125,Yerbogachen,RU,74.46,light rain,61.28,108.01,
132,Pontianak,ID,76.08,light rain,-0.03,109.33,
153,Grand-Santi,GF,73.38,light rain,4.25,-54.38,
158,Carroll,US,69.01,light rain,39.55,-77.02,
161,Gizo,SB,84.40,light rain,-8.10,156.84,


In [37]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [38]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""

In [41]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [48]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [43]:
vacation_df = pd.read_csv("weather_data/WeatherPy_Vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,5,Sangar,RU,73.29,light rain,63.92,127.47,NaN
1,20,Tecoanapa,MX,82.00,moderate rain,16.52,-98.75,Hotel Manglares
2,21,Hearst,CA,62.15,moderate rain,49.68,-83.67,Companion
3,29,Vangaindrano,MG,65.41,light rain,-23.35,47.60,Hotel Fanilo
4,91,Panzhihua,CN,82.15,light rain,26.59,101.71,Electric Power Hotel


AttributeError: 'Figure' object has no attribute 'savefig'